-------------------------------------------------------------
Libs e Preparação do ambiente Spark
-------------------------------------------------------------



In [1]:
# Java jdk utilitários 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
#Download do Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [3]:
# Importando a biblioteca os
import os

# Definindo a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Definindo a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
# Findspark
!pip install -q findspark

In [5]:
import findspark

# Iniciando o findspark
findspark.init()

In [6]:
# iniciar uma seção Spark
from pyspark.sql import SparkSession

# iniciando o spark context
spark = SparkSession.builder.master('local[*]').getOrCreate()
spark

# Usando DataFrame (DF)

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType
from datetime import date,datetime
from pyspark.sql import Row

In [ ]:
#-----------------
#Abrindo um CSV
#pyspark.rdd.RDD
#-----------------

#file= spark.sparkContext.textFile('/content/food_coded.csv')

#type(file) #pyspark.rdd.RDD


pyspark.rdd.RDD

In [ ]:
#-----------------
#Abrindo um CSV
#pyspark.sql.dataframe.DataFrame
#-----------------

#file=sc.read.csv('/content/food_coded.csv',header=True,sep=',',nanValue=None)


#type(file) #pyspark.sql.dataframe.DataFrame
#file.show()
#Classes de file

#file.printSchema()

In [12]:
#-----------------
#Criando um CSV do zero (em diferentes nós)
#pyspark.sql.dataframe.DataFrame
#-----------------

datf=spark.createDataFrame([
    Row(Cidade='Salvador', Populacao=6., Time='Bahia', Dat_cadastro=date(2021, 10, 10)),
    Row(Cidade='Sao Paulo',Populacao=45., Time='Sao Paulo', Dat_cadastro=date(2021, 5, 5)),
    Row(Cidade='Recife', Populacao=3., Time='Sport',Dat_cadastro=date(2021, 7, 2)),
    Row(Cidade='Maceio', Populacao=2., Time='CSA', Dat_cadastro=date(2021, 8, 1)),
])
#datf


datf.show()

+---------+---------+---------+------------+
|   Cidade|Populacao|     Time|Dat_cadastro|
+---------+---------+---------+------------+
| Salvador|      6.0|    Bahia|  2021-10-10|
|Sao Paulo|     45.0|Sao Paulo|  2021-05-05|
|   Recife|      3.0|    Sport|  2021-07-02|
|   Maceio|      2.0|      CSA|  2021-08-01|
+---------+---------+---------+------------+



In [17]:
# Exibindo tres registros DF na vertical
datf.show(3, vertical=True)
#ou 
#datf.collect()

-RECORD 0------------------
 Cidade       | Salvador   
 Populacao    | 6.0        
 Time         | Bahia      
 Dat_cadastro | 2021-10-10 
-RECORD 1------------------
 Cidade       | Sao Paulo  
 Populacao    | 45.0       
 Time         | Sao Paulo  
 Dat_cadastro | 2021-05-05 
-RECORD 2------------------
 Cidade       | Recife     
 Populacao    | 3.0        
 Time         | Sport      
 Dat_cadastro | 2021-07-02 
only showing top 3 rows



In [19]:
# Analise rápida  dos dados
datf.select("Cidade", "Populacao").describe().show()

+-------+---------+-----------------+
|summary|   Cidade|        Populacao|
+-------+---------+-----------------+
|  count|        4|                4|
|   mean|     null|             14.0|
| stddev|     null|20.73644135332772|
|    min|   Maceio|              2.0|
|    max|Sao Paulo|             45.0|
+-------+---------+-----------------+



In [20]:
# Exibindo o Schema do Dataframe
datf.printSchema()

root
 |-- Cidade: string (nullable = true)
 |-- Populacao: double (nullable = true)
 |-- Time: string (nullable = true)
 |-- Dat_cadastro: date (nullable = true)



In [22]:
# Carregando o modulo de funções, expecificamente a função Upper
from pyspark.sql.functions import upper 


datf.withColumn('Cidade_U', upper(datf.Cidade)).show() # cidades ficam em caixa alta em  Cidade_U

+---------+---------+---------+------------+---------+
|   Cidade|Populacao|     Time|Dat_cadastro| Cidade_U|
+---------+---------+---------+------------+---------+
| Salvador|      6.0|    Bahia|  2021-10-10| SALVADOR|
|Sao Paulo|     45.0|Sao Paulo|  2021-05-05|SAO PAULO|
|   Recife|      3.0|    Sport|  2021-07-02|   RECIFE|
|   Maceio|      2.0|      CSA|  2021-08-01|   MACEIO|
+---------+---------+---------+------------+---------+



In [24]:
#Select data

datf.filter(datf.Cidade == "Salvador").show()

+--------+---------+-----+------------+
|  Cidade|Populacao| Time|Dat_cadastro|
+--------+---------+-----+------------+
|Salvador|      6.0|Bahia|  2021-10-10|
+--------+---------+-----+------------+



In [25]:
# Criando uma tabela temporária em memória com os dados e utilizando consulta SQL


datf.createOrReplaceTempView("Dados")
spark.sql("select count(*) from Dados").show()

+--------+
|count(1)|
+--------+
|       4|
+--------+



## Converter RDD --> DF

In [31]:
# carregando os dados sobre Capital de paises
pais = [("Brasil",10000),("Argentina",20000),("Australia",35000),("Italia",40000),("Egito",65000),("Mexico",80000)]
rddpais= spark.sparkContext.parallelize(pais)


# convertendo em DF
dataframerdd= rddpais.toDF()
#type(dataframerdd) #pyspark.sql.dataframe.DataFrame


dataframerdd.show()

+---------+-----+
|       _1|   _2|
+---------+-----+
|   Brasil|10000|
|Argentina|20000|
|Australia|35000|
|   Italia|40000|
|    Egito|65000|
|   Mexico|80000|
+---------+-----+



In [32]:
# Criando o schema das colunas dos campos do DF
Colunas = ["Pais","Total_capital(Bilhoes)"]
dataframerdd2= rddpais.toDF(Colunas)
dataframerdd2.printSchema()
dataframerdd2.show(truncate=False)

root
 |-- Pais: string (nullable = true)
 |-- Total_capital(Bilhoes): long (nullable = true)

+---------+----------------------+
|Pais     |Total_capital(Bilhoes)|
+---------+----------------------+
|Brasil   |10000                 |
|Argentina|20000                 |
|Australia|35000                 |
|Italia   |40000                 |
|Egito    |65000                 |
|Mexico   |80000                 |
+---------+----------------------+

